In [1]:
import os
import json
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import web3

In [2]:
web3_geth = web3.Web3(web3.Web3.HTTPProvider("http://127.0.0.1:8546",request_kwargs={'timeout': 3600}))
web3_geth.isConnected()

True

In [3]:
web3_erigon = web3.Web3(web3.Web3.HTTPProvider("http://127.0.0.1:8545",request_kwargs={'timeout': 3600}))
web3_erigon.isConnected()

True

In [25]:
class Account:
    def __init__(self):
        self.account_balance_mainnet = []
        self.account_balance_fork = []
        self.first_block = 15537393
        self.last_mainnet_block = 15537420
        self.address = 0

### Filter all transaction where FTX is involved to extract relevant account balances

In [26]:
ftx_accounts = pd.read_csv("csv/ftx.csv")

In [27]:
account_list = ftx_accounts["accounts"].tolist()

In [28]:
account_balances = []

In [29]:
for index, address in enumerate(account_list):
    account = Account()
    account_balance_mainnet = []
    account_balance_fork = []
    for block_index_mainnet in range(15537393, 15537420):
        amount = web3_geth.eth.get_balance(web3_geth.toChecksumAddress(address), block_index_mainnet)
        account_balance_mainnet.append(amount)

    for block_index_fork in range(15537393, 15537420):
        amount = web3_erigon.eth.get_balance(web3_erigon.toChecksumAddress(address), block_index_fork)
        account_balance_fork.append(amount)
    account.account_balance_mainnet = account_balance_mainnet
    account.account_balance_fork = account_balance_fork
    account.last_mainnet_block = 15537420
    account.address = address
    account_balances.append(account)

In [ ]:
for account in account_balances:
    mainnet_balance_ether = [x / 10**18 for x in account.account_balance_mainnet]
    fork_balance_ether = [x / 10**18 for x in account.account_balance_fork]
    plt.figure(facecolor='white')
    plt.plot(mainnet_balance_ether, label="Mainnet", color="red")
    plt.plot(fork_balance_ether, label="Fork", color="blue")
    plt.legend()
    plt.xlabel("Block Range")
    plt.ylabel("Balance (in Ether)")

    fig = plt.gcf()
    fig.set_dpi(90)
    fig.set_figwidth(11)
    fig.set_figheight(6)
    fig.savefig("graphs/account_balance_ftx/" + str(account.address), dpi=300)